In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

In [9]:
# Load the preprocessed dataset
df_reviews = pd.read_csv('../Data/processed/df_reviews.csv')


In [14]:
# 2. Préparer les données
df = df_reviews
X = df['processed_text'].tolist()
y = df['sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2}).tolist()


In [15]:
# Diviser les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# 3. Charger le tokenizer et le modèle
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# 4. Fonction pour tokenizer les données
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128)

# 5. Convertir les données en format compatible
class YelpDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
# Tokeniser les données
train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)


In [19]:
# Créer les datasets
train_dataset = YelpDataset(train_encodings, y_train)
test_dataset = YelpDataset(test_encodings, y_test)


In [20]:
# 6. Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [21]:
# 7. Créer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [22]:
# 8. Entraîner le modèle
print("Début de l'entraînement...")
trainer.train()


Début de l'entraînement...


  0%|          | 0/25000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 9. Évaluer le modèle
print("\nÉvaluation du modèle:")
eval_results = trainer.evaluate()
print(f"Loss: {eval_results['eval_loss']}")


In [ ]:
# 10. Prédictions
print("\nPrédictions sur les données de test:")
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)


In [ ]:
# 11. Afficher les métriques de classification
from sklearn.metrics import classification_report
